PZFlow Informer and Estimator Demo

Author: Tianqing Zhang


In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import rail
from rail.core.data import TableHandle
from rail.core.stage import RailStage
import qp
import tables_io

from rail.estimation.algos.pzflow_nf import PZFlowInformer, PZFlowEstimator


In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
from rail.utils.path_utils import find_rail_file
trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
training_data = DS.read_file("training_data", TableHandle, trainFile)
test_data = DS.read_file("test_data", TableHandle, testFile)

In [ ]:
pzflow_dict = dict(hdf5_groupname='photometry',output_mode = 'not_fiducial' )


In [ ]:

# epoch = 200 gives a reasonable converged loss
pzflow_train = PZFlowInformer.make_stage(name='inform_pzflow',model='demo_pzflow.pkl',num_training_epochs = 30, **pzflow_dict)


In [ ]:
# training of the pzflow
pzflow_train.inform(training_data)

In [ ]:
pzflow_dict = dict(hdf5_groupname='photometry')

pzflow_estimator = PZFlowEstimator.make_stage(name='estimate_pzflow',model='demo_pzflow.pkl',**pzflow_dict, chunk_size = 20000)

In [ ]:
# estimate using the test data
estimate_results = pzflow_estimator.estimate(test_data)

In [ ]:
mode = estimate_results.read(force=True).ancil['zmode']
truth = np.array(test_data.data['photometry']['redshift'])

In [ ]:
# visualize the prediction. 
plt.figure(figsize = (8,8))
plt.scatter(truth, mode, s = 0.5)
plt.xlabel('True Redshift')
plt.ylabel('Mode of Estimated Redshift')